In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from model import Net

net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=18432, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=8, bias=True)
  (drop1): Dropout(p=0.25, inplace=False)
)


In [2]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [3]:
annotations = pd.read_csv('../data/AFLW/annotations.csv')

In [4]:
annotations.shape

(16418, 12)

In [5]:
annotations.rename(columns={"Unnamed: 0": "index"}, inplace=True)
annotations.head()

,index,face_id,LeftEye_x,LeftEye_y,RightEye_x,RightEye_y,Nose_x,Nose_y,Mouth_x,Mouth_y,file_id,sex
0,0,39341,209.0,150.0,302.0,186.0,277.000000,268.000000,232.000000,308.000000,image00035.jpg,m
1,1,39343,148.0,203.0,234.0,207.0,219.053909,272.376251,203.000000,308.000000,image00168.jpg,f
2,2,39348,143.0,123.0,223.0,173.0,159.983505,209.756149,134.000000,257.000000,image00102.jpg,f
3,3,39349,229.0,196.0,291.0,198.0,274.000000,218.000000,270.000000,254.000000,image00104.jpg,f
4,4,39352,334.0,229.0,526.0,249.0,486.200958,406.574982,433.417664,518.424988,image00122.jpg,f


In [6]:
from dotenv import load_dotenv
import os

load_dotenv()

dataset_path = os.getenv("DATA_PATH")

In [7]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class FaceKeypointDataset(Dataset):
    def __init__(self, annotations, root_dir, transform=None):
        
        self.annotations = annotations
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations["file_id"][idx])
        image = plt.imread(img_name)
        
        if len(image.shape) == 2:
            image = np.stack([image] * 3, axis=-1)

        landmarks = self.annotations.iloc[idx, 2:-2].values
        landmarks = landmarks.astype('float').reshape(-1, 2)

        if self.transform:
            image = self.transform(image)
            
        h, w = image.shape[1:]
        landmarks[:, 0] *= (224 / w)
        landmarks[:, 1] *= (224 / h)

        return image, torch.tensor(landmarks, dtype=torch.float32)


In [18]:
annotations.iloc[0, 2:10].values

array([np.float64(209.0), np.float64(150.0), np.float64(302.0),
       np.float64(186.0), np.float64(277.0), np.float64(268.0),
       np.float64(232.0), np.float64(308.0)], dtype=object)

In [19]:
landmarks

tensor([[[ 636.0000,  300.0000],
         [ 729.0000,  286.0000],
         [ 692.0000,  353.0000],
         [ 709.0000,  408.0000]],

        [[ 996.0000, 1380.0000],
         [1483.0000, 1309.0000],
         [1203.0000, 1718.0000],
         [1318.0000, 1830.0000]],

        [[ 337.0000,  142.0000],
         [ 374.0000,  139.0000],
         [ 358.0000,  158.0000],
         [ 359.0000,  178.0000]],

        [[ 211.0000,  174.0000],
         [ 256.0000,  179.0000],
         [ 226.0000,  198.0000],
         [ 227.0000,  215.0000]],

        [[ 371.0000,  115.0000],
         [ 415.0000,  117.0000],
         [ 393.0000,  135.0000],
         [ 393.0000,  158.0000]],

        [[1930.0000, 2024.0000],
         [2176.0000, 1700.0000],
         [2286.0000, 1969.0000],
         [2413.0000, 2111.0000]],

        [[ 742.0000,  262.0000],
         [ 846.0000,  298.0000],
         [ 850.0000,  358.0000],
         [ 789.0000,  438.0000]],

        [[ 140.0000,  237.0000],
         [ 174.0000,  221.000

In [9]:
os.path.join(dataset_path + "/annotated_imgs/", annotations["file_id"][5])

'//OMVICECAVE/nas/ml_data/AFLW//annotated_imgs/image00144.jpg'

In [10]:
# os.mkdir(dataset_path + "/annotaded_imgs/")
import shutil

# for dir in ["0", "2", "3"]:
#     for img in os.listdir(os.path.join(dataset_path, dir)):
#         if img in annotations['file_id'].values:
#             img_name = os.path.join(dataset_path, dir, img)
"""
# image = plt.imread(img_name)
# landmarks = annotations[annotations['file_id'] == img].values[0][1:-2].astype('float').reshape(-1, 2)
# plt.imshow(image)
# plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker='.', c='r')
# plt.savefig(dataset_path + "/annotaded_imgs/" + img)
# plt.close()
"""
            # shutil.copy(img_name, dataset_path + "/annotaded_imgs/" + img)
    # print(dir)

'\n# image = plt.imread(img_name)\n# landmarks = annotations[annotations[\'file_id\'] == img].values[0][1:-2].astype(\'float\').reshape(-1, 2)\n# plt.imshow(image)\n# plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker=\'.\', c=\'r\')\n# plt.savefig(dataset_path + "/annotaded_imgs/" + img)\n# plt.close()\n'

In [11]:
len(os.listdir(dataset_path + "/annotaded_imgs/"))

14280

In [12]:
annotations["file_id"].unique().shape

(14287,)

In [13]:
annotations["face_id"].unique().shape

(16418,)

In [14]:
dataset = FaceKeypointDataset(annotations, dataset_path + "/annotaded_imgs/", transform=transform)

In [15]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [16]:
for images, landmarks in dataloader:
    print(images.shape)
    print(landmarks.shape)
    break

torch.Size([32, 3, 224, 224])
torch.Size([32, 4, 2])


In [17]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net.to(device)

for epoch in range(10):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        labels = labels.view(labels.size(0), -1)

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        
        print(f"outputs shape: {outputs.shape}")
        print(f"labels shape: {labels.shape}")

        running_loss += loss.item()
        if i % 10 == 9:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 10}")
            running_loss = 0.0


outputs shape: torch.Size([32, 8])
labels shape: torch.Size([32, 8])
outputs shape: torch.Size([32, 8])
labels shape: torch.Size([32, 8])


KeyboardInterrupt: 